# This is a notebook used to perform spend analysis.
Using a comprehensive collection of spend and supplier data, this notebook sets to analyse the distribution of spend for a provided Class2 Code.
The user will be presented with a highover view of spend per Class3, per Group Vendor, and per Country of Origin. This notebook also contains an export element that combines order data with spend data, does clustering using K-means on purchase amount, and saves to a .xlsx file with columns ProductNumber, ProductDescription, SoldQuantity - all grouped per cluster. 

The insights provided by this notebook may be included when tendering assortment.

In [6]:
from IPython.display import display
import bigquery_connector

PROJECT_ID = "kramp-sharedmasterdata-prd"
DATASET_ID = "MadsH"
TABLE_ID = "purchase_data"

purchase_df = None

connector = None
if hasattr(bigquery_connector, "BigQueryConnector"):
    for kwargs in (
        {"project_id": PROJECT_ID, "dataset_id": DATASET_ID},
        {"project": PROJECT_ID, "dataset": DATASET_ID},
    ):
        try:
            connector = bigquery_connector.BigQueryConnector(**kwargs)
            break
        except TypeError:
            continue

if connector is not None:
    loader = next(
        (getattr(connector, attr) for attr in ("fetch_dataframe", "get_dataframe", "read_table", "load_table", "to_dataframe") if hasattr(connector, attr)),
        None,
    )
    if loader is not None:
        try:
            purchase_df = loader(TABLE_ID)
        except TypeError:
            for kwarg in ("table_id", "table", "name"):
                try:
                    purchase_df = loader(**{kwarg: TABLE_ID})
                    break
                except TypeError:
                    continue

if purchase_df is None:
    loader = next(
        (getattr(bigquery_connector, attr) for attr in ("fetch_dataframe", "get_dataframe", "read_table", "load_table", "to_dataframe") if hasattr(bigquery_connector, attr)),
        None,
    )
    if loader is not None:
        try:
            purchase_df = loader(
                project_id=PROJECT_ID,
                dataset_id=DATASET_ID,
                table_id=TABLE_ID,
            )
        except TypeError:
            for kwargs in (
                {"project": PROJECT_ID, "dataset": DATASET_ID, "table": TABLE_ID},
                {"project": PROJECT_ID, "dataset": DATASET_ID, "name": TABLE_ID},
            ):
                try:
                    purchase_df = loader(**kwargs)
                    break
                except TypeError:
                    continue

if purchase_df is None:
    raise AttributeError("No compatible loader found in bigquery_connector.py")

display(purchase_df.head())

ModuleNotFoundError: No module named 'bigquery_connector'